# Vítejte


* něco o mně...
* ...něco o Vás

## Co tenhle víkend probereme?

* funkce
* malinko nakousneme i funkce vyšších řádů ;)
* seznamy, iterátory, generátory
* dekorátory
* lambdy - anonymni funkce (viz bod 1)
* ...
* a kromě těchto nudných věcí si naprogramujeme plánovač letů - takové mini Kiwi :D

---

In [ ]:
from __future__ import annotations
import csv
import typing as t
from pprint import pprint


# XXX: Tohle nechte plavat, nekdy se k tomu treba vratime!
def yield_from_csv(csv_file: str) -> t.Iterable[dict]:
    """
    Open a `csv_file` and **yield** rows as a Python dictionary, ie. one-by-one.
    """
    with open(csv_file, newline="", encoding="utf-8") as f:
        for d in csv.DictReader(f):
            yield d

def list_from_csv(csv_file: str) -> list[dict]:
    """
    Open a `csv_file` and return a list of rows as Python dictionaries.
    """
    # Rovnou konzumujeme ten generator
    # return list(yield_from_csv(csv_file))
    
    result = []
    with open(csv_file, newline="", encoding="utf-8") as f:
        for d in csv.DictReader(f):
            result.append(d)

    return result

In [ ]:
# Nacteme zaznamy z `.csv`
flight_records = list_from_csv("flights.csv")

In [ ]:
pprint(flight_records)

In [ ]:
# Kolik jich vlastne je?
print(len(flight_records))

In [ ]:
first_record = list_from_csv("flights.csv")[0]

In [ ]:
print(first_record)

In [ ]:
# Vyhodi `TypeError`, protoze z `.csv` souboru nam to vylezlo jako retezec
print(first_record["price"] + 50)

In [ ]:
# Pokud bucyhom pouzili `csv.reader` misto `csv.DictReader`, dostali bychom to ve forme n-tic, se kterymi
# se sice da pracovat, ale po case byste nevedeli, jaky index patri k cemu, tj. ze 0-ty sloupec je `source`,
# 3. je `departure` a tak dale...
print(tuple(first_record.values()))

#### Typové anotace

```
def add(a: int, b: int) -> int:
    return a + b
```

Nejsou Pythonem vynucované (a tím pádem ani já je po vás nebudu chtit ;)), ale 
- zpřehledňují kód
- existují nástroje, které jsou schopné anotace analyzovat a říct vám, jestli nemáte v kódu chybu
- PyCharm vám bude líp napovídat a bude na vás svítit, pokud se mu něco nebude zdát ;)

Více zde: https://docs.python.org/3/library/typing.html

#### Iterátory

"funkce" `yield_from_csv` je zvláštní, protože vám nevrátí seznam, ale jakýsí "generátor" objekt -

In [ ]:
from_csv = yield_from_csv("flights.csv")
print(type(from_csv))

generátor nemá žádné prvky a hlavně se zatím neprovedla žádné čtení souboru - to se provede až při zavolání `next` na daný generátor.

In [ ]:
next(from_csv)

volat pořád `next` by bylo hodně neohrabané, takže funkce jako `list`, `tuple`, `set` zavolané na generátor ho
projedou až do konce.

In [ ]:
records_without_first = list(from_csv)

generátor ale **nemá** žadnou možnost jít dozadu, tj. žádná funkce `previous` není - k prkvům, které už jsme
prošli skrz `next` a neuložili si je, už nemáme přístup!

In [ ]:
first = next(from_csv)

To, že je generátor prazdný, poznáme podle toho, že se vyhodí speciální vyjímka - `StopIteration`

In [ ]:
print(len(records_without_first), len(flight_records))

#### K čemu teda jsou a jaké jsou jejich výhody a nevýhody?

- umožňují vám držet v paměti potenciálně nekonečné data
- práci nevykonáte hned, ale komponujete funkce nad daty, které pak vyhodnotíte, až je budete skutečně potřebovat

---

- seznamy prostě držíte v paměti a nemusíte si pamatovat, kolikrát jste je prošli
- narozdíl od seznamů neumožňují přístup podle indexu, napr. `my_list[5]`
- úplně nový koncept, který je obtížný na zpracování
- ...

---

### Úkol

Napište funkci, která vrátí všechny unikátní dvojice letišť, mezi kterými lítají letadla.

```
def unique_airport_combinations(flights):
    ...
```

Vracet to bude množinu všech dvojic, například:

```
{("DPS", "HKT"), ...}
```

V dokumentaci si můžete přečíst něco o tom, jak se používají množiny a n-tice:

* https://docs.python.org/3/library/stdtypes.html#set-types-set-frozenset
* https://docs.python.org/3/library/stdtypes.html#tuples


**BONUS** Modifikujte funkci tak, aby vracela nikoliv dvojice, ale řetězec tvaru `<source>-><destination>`, takže například

```
{"DPS->HKT", ...}
```

In [ ]:
def unique_flight_combinations_v1(flights: t.Iterable[dict]) -> set[tuple[str, str]]:
    result = []
    for flight in flights:
        result.append((flight["source"], flight["destination"]))

    return set(result)


def unique_flight_combinations_v2(flights: t.Iterable[dict]) -> set[tuple[str, str]]:
    result = set()
    for flight in flights:
        result.add((flight["source"], flight["destination"]))

    return result


def unique_flight_combinations_v3(flights: t.Iterable[dict]) -> set[tuple[str, str]]:
    # Kompaktni zapis pres set-comprehension
    # https://www.pythonforbeginners.com/basics/set-comprehension-in-python
    return {(flight["source"], flight["destination"]) for flight in flights}

In [ ]:
# Vsechny davaji stejny vysledek
print(unique_flight_combinations_v1(flight_records))
print(unique_flight_combinations_v2(flight_records))
print(unique_flight_combinations_v3(flight_records))

In [ ]:
def unique_flight_combinations_bonus_v1(flights: t.Iterable[dict]) -> set[str]:
    result = []
    for flight in flights:
        result.append(f"{flight['source']}->{flight['destination']}")

    return set(result)


def unique_flight_combinations_bonus_v2(flights: t.Iterable[dict]) -> set[str]:
    result = set()
    for flight in flights:
        result.add(f"{flight['source']}->{flight['destination']}")

    return result


def unique_flight_combinations_bonus_v3(flights: t.Iterable[dict]) -> set[str]:
    # Kompaktni zapis pres set-comprehension
    # https://www.pythonforbeginners.com/basics/set-comprehension-in-python
    return {f"{flight['source']}->{flight['destination']}" for flight in flights}

In [ ]:
# Vsechny davaji stejny vysledek
print(unique_flight_combinations_bonus_v1(flight_records))
print(unique_flight_combinations_bonus_v2(flight_records))
print(unique_flight_combinations_bonus_v3(flight_records))

### Povídání o rozdílech mezi seznamem, množinou, slovníkem...

In [ ]:
my_list = [1, 2, 3, 4, 1, 1, 2]

In [ ]:
# seznam jde indexovat
print(my_list[2])

In [ ]:
# `set` jde zavolat na `list`, abyste se zbavili duplikatu.
print(set([1, 2, 3, 4, 1, 1, 2]))

In [ ]:
# Konstrukce mnoziny sama o sobe se taky zbavi duplikatu :) 
my_set = {5, 1, 2, 3, 4, 1, 1, 2}

In [ ]:
print(my_set)

In [ ]:
# Mnozina nejde indexovat - vyhodi to `TyperError`
my_set[0]

In [ ]:
my_dict = {"a": 1, "b": 2}

In [ ]:
# Ke klicum ve slovniku pristupujeme pres hranate zavorky
print(my_dict["a"])

In [ ]:
# Uzitecna vlastnost mnoziny - extremne rychly test, zda dany prvek v mnozine je ci ne
print(1 in my_set)
print(420 in my_set)

In [ ]:
# Odeber nahodny prvek z mnoziny a dej ho do promenne `random_element`
random_element = my_set.pop()

In [ ]:
print(my_set)

In [ ]:
print(random_element)

In [ ]:
# Muzeme predelat mnozinu na seznam - pak uz muzeme indexovat
elements = list(my_set)

In [ ]:
print(elements)
print(elements[0])

In [ ]:
# Jaky je rozdil mezi
# my_dict["a"] a my_dict.get("a") ?
print(my_dict["a"])
print(my_dict.get("a"))

# V pripade, kdy klic ve slovniku je, zadny rozdil

In [ ]:
# Vyhodi `KeyError` - klic `abc` neni ve slovniku!
print(my_dict["abc"])

In [ ]:
# `.get` v pripade, ze neni klic neni ve slovniku, vraci `None`.
print(my_dict.get("abc"))

In [ ]:
# Muzeme to potunit i druhym, volitelnym argument, kterym dame najevo, jakou hodnotu ma `.get` vratit, kdyz
# tam klic neni
print(my_dict.get("abc", "Nic tu neni!"))

### Úkol

Naimplementujte funkci `parse_flight_info`, která bude brát slovník, který jsme dostali z toho `csv`,
a vrátí taktéž slovník, který už ale nebude mít cenu a jiné položky jako řetězce a datum bude pěkně v `datetime` objektu.

tj. slovník

```
{'source': 'USM',
 'destination': 'HKT',
 'departure': '2017-02-11T06:25:00',
 'arrival': '2017-02-11T07:25:00',
 'flight_number': 'PV404',
 'price': '24',
 'bags_allowed': '1',
 'bag_price': '9'}
```

bude po transformaci vypadat následovně

```
{'source': 'USM',
'destination': 'HKT',
'departure': datetime.datetime(2017, 2, 11, 6, 25),
'arrival': datetime.datetime(2017, 2, 11, 7, 25),
'flight_number': 'PV404',
'price': 24,
'bags_allowed': 1,
'bag_price': 9}
```

Tj. 
```
def parse_flight_info(d: dict) -> dict:
    # Vase implementace zde
```

Možná se vám bude hodit funkce `datetime.fromisformat` - https://docs.python.org/3/library/datetime.html#datetime.datetime.fromisoformat


**BONUS** Vytvořte další funkce `yield_parsed_from_csv` a `list_parsed_from_csv`, které budou brát 2 parametry -
`parser` a `csv_file`.

- `parser` bude funkce (ano, můžeme předávat funkci jako parametr jinou funkci!)
- `csv_file` bude mít význám jako původně

které budou dělat to stejné, jako jejich nemodifikované předlohy `yield/list_from_csv`, jen všechny prvky už budou prohnané skrz ten `parser`

```
def yield_parsed_from_csv(parser, csv_file: str):
    ...
```

#### Honzova implementace

funkcni, ale operuje na celem seznamu - spis bych chtel funkce, ktera bere 1 prvek

```
def parse_flight_info(flights):

    for flight in flights:
        flight["departure"] = datetime.fromisoformat(flight.get("departure"))
        flight["arrival"] = datetime.fromisoformat(flight.get("arrival"))
        flight["price"] = int(flight.get("price"))
        flight["bags_allowed"] = int(flight.get("bags_allowed"))
        flight["bag_price"] = int(flight["bag_price"])
    return flights
```

#### Schematicky zapis

- vsimnete si toho mnozneho cisla 
```
def parse_flight_infos(flights):
    result = []
    for flight in flights:
        # Kazdy zaznam transformuj
        parsed = parse_flight_info(flight)
        result.append(parsed)

    # Vrat vsechny transformovane
    return result
```

In [ ]:
# Implementace ukolu
def parse_flight_info(raw: dict) -> dict:
    return {
        "source": raw["source"],
        "destination": raw["destination"],
        "flight_number": raw["flight_number"],
        "price": int(raw["price"]),
        "bags_allowed": int(raw["bags_allowed"]),
        "bag_price": int(raw["bag_price"]),
        "departure": datetime.fromisoformat(raw["departure"]),
        "arrival": datetime.fromisoformat(raw["arrival"])
    }

In [ ]:
# Kdyz napiseme funkci bez zavorek, je to pythoni objekt typu `function`
print(parse_flight_info)
print(type(parse_flight_info))

In [ ]:
# Pro pripomentu - `first_record` je 1. radek v csv souboru

# nezpracovany
pprint(first_record)

#
print("\n")
#

# zpracovany - vsimente si toho `datetime` objektu a i cisel misto retezcu
pprint(parse_flight_info(first_record))

In [ ]:
# Implementace bonusoveho ukolu - na `yield_` variantu zapomente :D
def list_parsed_from_csv(      
    parser_function: t.Callable[[dict], dict],                                              
    csv_file: str,                                                                          
) -> list[dict]:
    # Vytahneme prvne nezpracovane zaznamy z `.csv`
    unparsed_records = list_from_csv(csv_file)                                                                          
                                                                                                               
    result = []                                                                                                            
    for record in unparsed_records:
        # Na kazdy zaznam aplikujeme tu fuknci `parser_function`
        parsed = parser_function(record)
        result.append(parsed)                    
                                                                                                                 
    return result

In [ ]:
parsed_records = list_parsed_from_csv(parse_flight_info, csv_file="flights.csv")

In [ ]:
# `datetime` jsou na miste, stejne jako cisla :)
# Kuprikladu posledni zaznam
pprint(parsed_records[-1])

### Funkce jako parametry jiných funkcí?!

Už jste se s tím setkali - například funkce `max`, `min`, `sorted` berou jako nepovinný parametr `key`, což je funkce, která určuje řadící kritérium

In [ ]:
# pouzivame vychozi razeni - abecedne
print(max(["avocado", "apple", "banana", "peach", "pineapple"]))

In [ ]:
# radime podle delky toho retezce
print(max(["avocado", "apple", "banana", "peach"], key=len))

In [ ]:
# muzeme radit i podle neobvyklych kriterii, napriklad posledni pismeno daneho slova :D
print(max(["avocado", "apple", "banana", "peach", "pineapple"], key=lambda x: x[-1]))

In [ ]:
# Tohle vyhodi `TypeError`, protoze nejde porovnavat 2 slovniky
print(max([{"value": 5}, {"value": 1}, {"value": 3}]))

In [ ]:
# `key` argument nas zachrani!
# Vrati to prvek, ktery ma nejvyssi hodnotu u klice `value`
print(max([{"value": 5}, {"value": 1}, {"value": 3}], key=lambda x: x["value"]))

### Úkol

Napište funkce, které budou jako parametr brát seznam letů a vrátí

- kolik stál nejdražší let
- kolik stál nejlevnější let
- jak byl dlouhý nejdelší let
- kolik bylo nejvíce povolených zavazadel

Například:

```
def longest_flight_duration(flights) -> int:
    # Vase implementace zde
```

PS: Nemusíte jít popořadě a zároveň se nemusíte stresovat, že nemáte všechny

### Úkol

Napište funkci, která bude brát jako parametr let a číslo vyjadřující počet zavazadel a vrátí 
`True/False` podle toho, jestli je umožněno mít s sebou tolik zavazadel na daném letu.


```
def number_of_bags_allowed(flight: dict, bags: int) -> bool:
    # Vase implementace zde
```

### Úkol

Napište funkci, která bude brát jako parametr let a číslo vyjadřující počet zavazadel a vrátí 
celkovou cenu letenky.

Využijte k tomu již vytvořenou funkci `number_of_bags_allowed`. Pokud daný počet zavazadel není umožněn,
vyhoďte vyjímku `ValueError("Invalid number of bags!")`


```
def number_of_bags_allowed(flight: dict, bags: int) -> bool:
    # Vase implementace zde
```